In [6]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:85% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="6" color="red">ch14. 웹데이터 수집</font></b>
# 1절. BeautifulSoup과 parser


In [2]:
pip show bs4

Note: you may need to restart the kernel to use updated packages.


pip install bs4 아나콘다를 설치하면 자동 설치되는 패키지 7500개에 포함

- 공식 Documentation : https://beautiful-soup-4.readthedocs.io/en/latest/

In [4]:
# 내 local PC의 파일을 url(http://~)처럼 접근
import requests # http 요청 처리하는 lib
from requests_file import FileAdapter # file://프로토콜을 다루기 위함

In [5]:
s = requests.Session() # HTTP 요청처리하는 세션 객체
s.mount("file://", FileAdapter())
# file://경로로 들어오면 이 요청은 c:(로컬파일)을 접근
response = s.get('file:///ai/source/01_python/data/ch14_sample.html')
response

<Response [200]>

In [7]:
if response:
    print('잘 접근하였습니다.')
else:
    print('접근을 못했습니다')

잘 접근하였습니다.


In [8]:
response.status_code
    # 200 : 정상
    # 404 : 없는페이지
    # 406 : get, post 오류

200

In [14]:
response.content # 바이너리 형식의 html내용

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n  <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n  <div class="contents">\r\n    \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n    <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span>\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n  </div>\r\n  <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\

In [15]:
response.content.decode('utf-8')

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [9]:
response.text # HTML파일의 텍스트 내용

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [13]:
# html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, # response.text
                    'html.parser')
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
</head>
<body>
<h1 class="greeting css" id="text">Hello, CSS</h1>
<h1 class="css">Hi, CSS</h1>
<div id="subject">subject 선택자 안의 내용</div>
<p>CSS 선택자는 다양한 곳에서 활용됩니다</p>
<div class="contents">
    선택자를 어떻게 작성하느냐에 따라
    <span>다른<b>요소가 반환</b></span>됩니다
  </div>
<div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>
</body>
</html>

In [19]:
# soup.select_one('선택자') : 해당선택자 처음 하나만
el = soup.select_one('h1') # 처음 나오는 h1태그 하나만
print('el :', el)
print('el.text :', el.text)
print('el.string :', el.string)
print('el의 속성들 :', el.attrs)
print('el의 id속성 :', el.attrs['id']) # el.attrs은 딕셔너리
print('el의 id속성 :', el.attrs.get('id'))
print('el의 href속성 :', el.attrs.get('href'))
print('el의 태그이름 :', el.name)

el : <h1 class="greeting css" id="text">Hello, CSS</h1>
el.text : Hello, CSS
el.string : Hello, CSS
el의 속성들 : {'class': ['greeting', 'css'], 'id': 'text'}
el의 id속성 : text
el의 id속성 : text
el의 href속성 : None
el의 태그이름 : h1


In [20]:
# soup.select('선택자') : 해당 선택자 모두 
els = soup.select('h1') # h1태그를 list
print('els :', els)
print('els의 text들 :', [el.text for el in els])
print('els의 string들 :', [el.string for el in els])
print('els의 속성들 :', [el.attrs for el in els])

els : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
els의 text들 : ['Hello, CSS', 'Hi, CSS']
els의 string들 : ['Hello, CSS', 'Hi, CSS']
els의 속성들 : [{'class': ['greeting', 'css'], 'id': 'text'}, {'class': ['css']}]


In [22]:
# soup.find(태그, 속성) : soup.select_one('선택자')와 유사
print('select_one :', soup.select_one('h1.css') )
print('find :', soup.find('h1', {'class':'css'}) ) # soup.select_one('h1.css')
print('find :', soup.find('h1', class_='css') )
print()
print('select_one :', soup.select_one('h1#text') )
print('find :', soup.find('h1', {'id':'text'}))

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>


In [23]:
# soup.find_all(태그, 속성) : soup.select('선택자')와 유사
print('select :', soup.select('h1.css'))
print('find_all :', soup.find_all('h1', class_='css'))
print('find_all :', soup.find_all('h1', {'class':'css'}))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [24]:
print('select :', soup.select('h1.css, span'))
print('find_all :', soup.find_all(['h1', 'span'], {'class':'css'}))
print('find_all :', soup.find_all('h1', class_='css') + \
                     soup.find_all('span'))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]


In [25]:
# 없는 엘리먼트 찾기
print('select(빈 list) :', soup.select('a.css'))
print('select_one(None) :', soup.select_one('a.css'))
print('find_all(빈 list) :', soup.find_all('a', {'class', 'css'}))
print('find(None) :', soup.find('a', {'class':'css'}))

select(빈 list) : []
select_one(None) : None
find_all(빈 list) : []
find(None) : None


# 2절. 정적 웹 데이터 수집(정적 웹크롤링)
## 2.1 BeautifulSoup을 활용한 html 웹 데이터수집
### 1) 환율정보 가져오기(네이버증권 > 시장지표)
 - https://finance.naver.com/marketindex/

 크롤링 허용범위는 사이트마다 ~/robots.txt에서 확인
 
    # Allow : / - 사이트의 모든 경로(/)에 대한 크롤링 허용

In [31]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex/'
response = requests.get(url)
# response.status_code
# response.text / response.content.decode('cp949')
soup = BeautifulSoup(response.text, "html.parser")


<script language="javascript" src="/template/head_js.naver?referer=info.finance.naver.com&amp;menu=marketindex&amp;submenu=market"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20251105171339/js/info/jindo.min.ns.1.5.3.euckr.js" type="text/javascript"></script>
<script src="https://ssl.pstatic.net/imgstock/static.pc/20251105171339/js/jindo.1.5.3.element-text-patch.js" type="text/javascript"></script>
<div id="container" style="padding-bottom:0px;">
<div class="market_include">
<div class="market_data">
<div class="market1">
<div class="title">
<h2 class="h_market1"><span>환전 고시 환율</span></h2>
</div>
<!-- data -->
<div class="data">
<ul class="data_lst" id="exchangeList">
<li class="on">
<a class="head usd" href="/marketindex/exchangeDetail.naver?marketindexCd=FX_USDKRW" onclick="clickcr(this, 'fr1.usdt', '', '', event);">
<h3 class="h_lst"><span class="blind">미국 USD</span></h3>
<div class="head_info point_dn">
<span class="value">1,454.50</span>
<span class="txt_krw

In [32]:
# 방법2
from urllib.request import urlopen
url = 'https://finance.naver.com/marketindex/'
response = urlopen(url)
response.status
# response.read() / response.read().decode('cp949')
soup = BeautifulSoup(response, "html.parser")

In [36]:
# div.head_info > span.value (find함수)
prices = []
headinfos = soup.find_all('div', class_='head_info')
for headinfo in headinfos:
    price = headinfo.find('span', class_='value')
    # print(price.text.replace(',', ''))
    prices.append(float(''.join(price.text.split(',') ) ) )
print(prices)

[1455.3, 945.58, 1682.11, 204.34, 153.42, 1.1564, 1.3161, 99.47, 59.75, 1698.27, 4009.8, 189405.9]


In [38]:
# div.head_info > span.value (select함수)
prices = soup.select('div.head_info > span.value')
print([float(p.text.replace(',','')) for p in prices])

['1455.30',
 '945.58',
 '1682.11',
 '204.34',
 '153.4200',
 '1.1564',
 '1.3161',
 '99.4700',
 '59.75',
 '1698.27',
 '4009.8',
 '189405.9']

In [39]:
titles = soup.select('h3.h_lst > span.blind')
for t in titles:
        print(t.text, end='\t')
#print([t.text+'\t' for t in titles])

미국 USD	일본 JPY(100엔)	유럽연합 EUR	중국 CNY	달러/일본 엔	유로/달러	영국 파운드/달러	달러인덱스	WTI	휘발유	국제 금	국내 금	

In [40]:
units = soup.select('div.head_info > span > span.blind')
units = [unit.text for unit in units]
units.insert(7, '') # 7번째 값이 없어서 '' 추가
units

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [42]:
statuses = soup.select('div.head_info > span.blind')
for idx in range(len(statuses)):
    print(statuses[idx].text, end='\t')

하락	하락	하락	하락	상승	하락	상승	하락	상승	상승	상승	상승	

In [43]:
len(titles), len(prices), len(units), len(statuses)

(12, 12, 12, 12)

In [44]:
for idx in range(len(titles)):
    print("{} : {}{} - {}".format(titles[idx].text,
                                 prices[idx].text,
                                 units[idx],
                                 statuses[idx].text))

미국 USD : 1,455.30원 - 하락
일본 JPY(100엔) : 945.58원 - 하락
유럽연합 EUR : 1,682.11원 - 하락
중국 CNY : 204.34원 - 하락
달러/일본 엔 : 153.4200엔 - 상승
유로/달러 : 1.1564달러 - 하락
영국 파운드/달러 : 1.3161달러 - 상승
달러인덱스 : 99.4700 - 하락
WTI : 59.75달러 - 상승
휘발유 : 1698.27원 - 상승
국제 금 : 4009.8달러 - 상승
국내 금 : 189405.9원 - 상승


In [45]:
for title, price, unit, status in zip(titles, prices, units, statuses):
    print("{} : {}{} -{}".format(title.text,
                                price.text,
                                unit,
                                status.text))

미국 USD : 1,455.30원 -하락
일본 JPY(100엔) : 945.58원 -하락
유럽연합 EUR : 1,682.11원 -하락
중국 CNY : 204.34원 -하락
달러/일본 엔 : 153.4200엔 -상승
유로/달러 : 1.1564달러 -하락
영국 파운드/달러 : 1.3161달러 -상승
달러인덱스 : 99.4700 -하락
WTI : 59.75달러 -상승
휘발유 : 1698.27원 -상승
국제 금 : 4009.8달러 -상승
국내 금 : 189405.9원 -상승


### 2) 이번주 로또 번호 출력
 - https://dhlottery.co.kr/gameResult.do?method=byWin (google에 로또번호 당첨번호 검색)
 
 1197회(2025년 11월 08일 추첨)
 
 당첨번호 [1, 5, 7, 26, 28, 43]
 
 보너스 30

In [50]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
response.status_code
Lotto = BeautifulSoup(response.text, "html.parser")
Lotto


<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="동행복권" id="utitle" name="title"/>
<meta content="동행복권 1197회 당첨번호 1,5,7,26,28,43+30. 1등 총 13명, 1인당 당첨금액 2,205,089,827원." id="desc" name="description"/>
<title>로또6/45 - 회차별 당첨번호</title>
<title>동행복권</title>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="/images/common/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/images/common/favicon.ico" rel="icon" type="image/x-icon"/>
<script src="/js/jquery-1.9.1.min.js" type="text/javascript"></script>
<script src="/js/jquery-ui.js" type="text/javascript"></script>
<script charset="utf-8" src="/js/common.js" type="text/javascript"></script>
<script type="text/javascript">

fn_g_init_message("");

var gameUserId = "";

function goGame(){
	var userId = "";
	
	if(userId == '' || userId == null){
		alert("로그인 후 사용 해주시기 바랍니다.");
		location.href = "/user.do?method=login";
		return;
	}
	
	$.ajax({
		type:"get",                          

In [60]:
nlottos = Lotto.select('div.win_result > h4 strong')
[nlott.text for nlott in nlottos]

['1197회']

In [65]:
tlottos = Lotto.select('div.win_result > p.desc')
[tlott.text for tlott in tlottos]

['(2025년 11월 08일 추첨)']

In [91]:
#  > span.ball_645
numN = Lotto.select('div.nums > div.num > p > span.ball_645')
[ilott.text for ilott in numN]

['1', '5', '7', '26', '28', '43', '30']

In [84]:
Lotto.select('div.num')

[<div class="num win">
 <strong>당첨번호</strong>
 <p>
 <span class="ball_645 lrg ball1">1</span>
 <span class="ball_645 lrg ball1">5</span>
 <span class="ball_645 lrg ball1">7</span>
 <span class="ball_645 lrg ball3">26</span>
 <span class="ball_645 lrg ball3">28</span>
 <span class="ball_645 lrg ball5">43</span>
 </p>
 </div>,
 <div class="num bonus">
 <strong>보너스</strong>
 <p><span class="ball_645 lrg ball3">30</span></p>
 </div>]

In [92]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
#soup

In [93]:
# 방법2
from urllib.request import urlopen
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = urlopen(url)
response.status
soup = BeautifulSoup(response, "html.parser")
# soup

In [94]:
times = soup.select_one('div.win_result strong').text # 1197회
date  = soup.select_one('div.win_result > p.desc').text
# date = soup.find('p', class_='desc').text
lotto_number_el = soup.select('div.num.win span')
lotto_number = [int(el.text) for el in lotto_number_el]
bonus_number = int(soup.select_one('div.num.bonus > p > span').text)

In [95]:
print(times, date)
print('당첨번호 ',lotto_number)
print('보 너 스 ', bonus_number)

1197회 (2025년 11월 08일 추첨)
당첨번호  [1, 5, 7, 26, 28, 43]
보 너 스  30


### 3) 다음 검색 리스트
```
no title               href
0  "한풀 꺾인 비트코인  https://v.daum.net/v/20251110094711892
```

In [102]:
# 방법1
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# 방법2 : 불가
# from urllib.request import urlopen
# from urllib.parse import quote
# word = quote('비트코인') # url에 한글이 포함된 경우 한글을 quote()로 변환
# url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
# print(url)
# response = urlopen(url)
# soup = BeautifulSoup(response, "html.parser")

items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    # print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx,
                           'title':item.text,
                           'link':item.attrs.get('href')})
import pandas as pd
pd.DataFrame(items_find_list)

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=비트코인


,no,title,link
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
6,6,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
7,7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [103]:
items_find_list = [] # 검색한 결과를 담을 리스트
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    items_find_list.append([idx, item.text, item.attrs['href']])
pd.DataFrame(items_find_list, columns=['번호', '제목', '링크'])

,번호,제목,링크
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
6,6,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
7,7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [104]:
items_find_list = [] # 검색한 결과를 담을 리스트
# div.item-title > strong.tit-g > a
item_titles = soup.find_all('div', class_='item-title')
for idx, item in enumerate(item_titles):
    a = item.find('a')
    #print(idx, a.text, a.attrs['href'])
    #items_find_list.append([idx, a.text, a.attrs['href']])
    items_find_list.append({
        'no':idx,
        'title':a.text,
        'link':a.attrs['href'],
    })
pd.DataFrame(items_find_list)

,no,title,link
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
6,6,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
7,7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [120]:
# 다음 뉴스 검색(원하는 키워드를 원하는 페이지를 가져오기)
import requests
from bs4 import BeautifulSoup
import pandas as pd
def collect_news_list(keyword, page):
    'keyword로 page에 다음 뉴스검색한 결과를 출력->list를 return'
    #url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={page}'
    # response = requests.get(url)
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    items_find_list = [] # 검색한 결과를 담는 리스트
    item_titles = soup.select('div.item-title > strong.tit-g > a')
    for idx, item in enumerate(item_titles):
        items_find_list.append([(page-1)*10+idx, item.text, item.attrs['href']])
    return items_find_list

In [106]:
collect_news_list('비트코인', 3)

비트코인
[20, ' 비트코인, 위태로운 10만 달러..."하락장 전환"vs"숨고르기" ', 'http://v.daum.net/v/20251106140714590']
[21, ' 연구소서 실험하려 샀는데…비트코인 1000배 대박 터졌다 ', 'http://v.daum.net/v/20251109120511705']
[22, ' "비트코인 \'1000배 잭팟\' 터뜨렸어요"···실험용으로 97개 샀다가 \'횡재\' 맞은 \'이 나라\' ', 'http://v.daum.net/v/20251109075014647']
[23, ' 돈나무 언니도 생각 바꿨다…비트코인 강세전망 20%↓ ', 'http://v.daum.net/v/20251107040000760']
[24, ' JP모건 "비트코인, 금 비해 저평가...공정가치 17만 달러" ', 'http://v.daum.net/v/20251107092549359']
[25, ' 돈나무 언니, 비트코인 전망 120만달러로 낮췄다...왜? ', 'http://v.daum.net/v/20251107170008576']
[26, " 연구용으로 산 비트코인 97개…13년만에 1000배 수익 '잭팟' ", 'http://v.daum.net/v/20251109113613079']
[27, ' “주식보다 더 떨어져”…휘청이는 비트코인 “분위기 반전 어렵다” ', 'http://v.daum.net/v/20251106171049044']
[28, ' 한때 10만弗 붕괴, 날개 꺾인 비트코인 ', 'http://v.daum.net/v/20251106041647056']
[29, ' “곡소리 들리네” 비트코인·이더리움 붕괴…위험자산 ‘회피 심리’ ', 'http://v.daum.net/v/20251105080314455']


In [121]:
import time
keywords = ['청바지', '동대문']
result0 = [] # 청바지 1~3페이지까지 검색한 결과를 담을 list
result1 = [] # 동대문 1~3페이지까지 검색한 결과를 담을 list
pages = 3
for i, keyword in enumerate(keywords):
    print(f'==={i+1}번째 검색어 {keyword} 검색 결과 수집 중입니다===')
    for page in range(1, pages+1):
        if i==0:
            result0.extend(collect_news_list(keyword, page))
        elif i==1:
            result1.extend(collect_news_list(keyword, page))
        time.sleep(0.5)

===1번째 검색어 청바지 검색 결과 수집 중입니다===
===2번째 검색어 동대문 검색 결과 수집 중입니다===


In [124]:
result0_df = pd.DataFrame(result0, columns=['no', 'title','link'])
result1_df = pd.DataFrame(result1, columns=['no', 'title','link'])
result0_df
result1_df

,no,title,link
0,0,"“돈 벌어가세요” 동대문 일요시장, 겨울옷 핫플된 사연? [르포]",http://v.daum.net/v/20251110100357735
1,1,"동대문구, ‘11월 자전거교실’ 개강… 올해 마지막 초보자 교육 진행",http://v.daum.net/v/20251110145114101
2,2,"음주운전 차, 동대문역서 인도 돌진...日 관광객 '사망'",http://v.daum.net/v/20251103073316480
3,3,"동대문구, 25일까지 '주민 건강도시학교'···""건강하게 살자""",http://v.daum.net/v/20251109223318356
4,4,"서울 동대문구, 15일 배봉산정원 가족축제 개최",http://v.daum.net/v/20251104104517122
5,5,"동대문구, 관내 144명 학생에 1억3670만원 장학금 지급",http://v.daum.net/v/20251107161116880
6,6,"동대문구, 청량리시장 1·2공영주차장까지 무인정산 확대",http://v.daum.net/v/20251107073911464
7,7,"동대문문화재단, ‘찾아가는 문화행사’ 성황 개막",http://v.daum.net/v/20251106064448976
8,8,"'해외연수비 과다청구' 의혹 동대문구의회 의장, 사과 요구하자 ""알아서 하겠다""",http://v.daum.net/v/20251106174500454
9,9,"동대문구, ‘2025 안전심포지엄’ 개최…실전 대응력 점검",http://v.daum.net/v/20251105150913954


In [123]:
result0_df.to_csv(f'data/ch14_{keywords[0]}.csv', index=False, encoding='cp949')
result1_df.to_csv(f'data/ch14_{keywords[1]}.csv', index=False)

### 4) User-Agent를 추가하여 크롤링
 - 자동으로 브라어저를 통해 요청하는 것 처럼 보이기

In [127]:
# 방법2 : 
from urllib.request import urlopen, Request
from urllib.parse import quote
word = quote('비트코인') # 한글을 encoding 처리
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q='+word
print(url)
# headers = {'User-Agent':
#           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
# request = Request(url, headers=headers)
request = Request(url)
request.add_header('User-Agent', 
                'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36')
response = urlopen(request)
soup = BeautifulSoup(response, 'html.parser')
# soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


 - https://www.melon.com/chart/index.htm
  - https://wwww.melon.com/robots.txt 에서 User-Agent에 봇이 지정

In [128]:
# 방법1
import requests
url = 'https://www.melon.com/chart/index.htm'
melonpage = requests.get(url)
melonpage
# soup = BeautifulSoup(melonpage.text, 'html.parser')
# soup

<Response [406]>

In [130]:
# 방법2
from urllib.request import urlopen, Request
url = 'https://www.melon.com/chart/index.htm'
# melonpage = urlopen(url) # 에러남
headers = {'user-agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36'}
request = Request(url, headers=headers)
melonpage = urlopen(request)
soup = BeautifulSoup(melonpage, 'html.parser')
#soup

<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<title>멜론차트&gt;TOP100&gt;멜론</title>
<meta content="음악서비스, 멜론차트, 멜론TOP100, 최신음악, 인기가요, 뮤직비디오, 앨범, 플레이어, 스트리밍, 다운로드, 아티스트플러스, 아티스트채널" name="keywords"/>
<meta content="No.1 뮤직플랫폼 멜론! 최신 트렌드부터 나를 아는 똑똑한 음악추천까지!" name="description"/>
<meta content="ee85ff6db1fa8f2226bcb671ecb2bcdbcffb6f8b" name="naver-site-verification"/>
<meta content="q4tbTQhmxa4La3OdNLsNOCxrJ_WV6lUlBFrFW4-HqQc" name="google-site-verification"/>
<meta content="4022717807957185" property="fb:app_id"/>
<meta content="Melon" property="og:title"/>
<meta content="https://cdnimg.melon.co.kr/resource/image/web/common/logo_melon142x99.png" property="og:image"/>
<meta content="음악이 필요한 순간, 멜론" property="og:description"/>
<meta content="http://www.melon.com/chart/index.htm" property="og:url"/>
<meta content="website" property="og:type"/>
<meta content="멜론" prope

In [133]:
# 방법1 : User-Agent 추가
url = 'https://www.melon.com/chart/index.htm'
headers = {'user-agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, # melonpage.content
                    'html.parser')

ranks = soup.select('td div.wrap.t_center > span.rank')
titles = soup.select('div.ellipsis.rank01 > span') # title.text.strip()
singers = soup.select('div.ellipsis.rank02') # singer.text.strip()[:20]
# 1위 | 노래제목 - 가수명
for rank, title, singer in zip(ranks, titles, singers):
    print(f'{rank.text}위 | {title.text.strip()} - {singer.text.strip()[:20]}')

1위 | Blue Valentine - NMIXXNMIXX
2위 | 타임캡슐 - 다비치다비치
3위 | Golden - HUNTR/X, EJAE, AUDRE
4위 | Drowning - WOODZWOODZ
5위 | Good Goodbye - 화사 (HWASA)화사 (HWASA)
6위 | SPAGHETTI (feat. j-hope of BTS) - LE SSERAFIM (르세라핌), 
7위 | 달리 표현할 수 없어요 - 로이킴로이킴
8위 | 뛰어(JUMP) - BLACKPINKBLACKPINK
9위 | Soda Pop - KPop Demon Hunters C
10위 | 어제보다 슬픈 오늘 - 우디 (Woody)우디 (Woody)
11위 | FAMOUS - ALLDAY PROJECTALLDAY
12위 | 시작의 아이 ❍ - 박다혜, 마크툽 (MAKTUB)박다혜
13위 | IRIS OUT - Kenshi YonezuKenshi 
14위 | XOXZ - IVE (아이브)IVE (아이브)
15위 | 모르시나요(PROD.로코베리) - 조째즈조째즈
16위 | Rich Man - aespaaespa
17위 | Hollywood Action - BOYNEXTDOORBOYNEXTDO
18위 | 한번 더 이별 - 이창섭이창섭
19위 | 내게 사랑이 뭐냐고 물어본다면 - 로이킴로이킴
20위 | 너에게 닿기를 - 10CM10CM
21위 | HOME SWEET HOME (feat. 태양, 대성) - G-DRAGONG-DRAGON
22위 | 운명 (2025) - 먼데이 키즈, 이이경먼데이 키즈, 이
23위 | body - 다영 (DAYOUNG)다영 (DAYO
24위 | toxic till the end - 로제 (ROSÉ)로제 (ROSÉ)
25위 | Whiplash - aespaaespa
26위 | 시작의 아이 - 마크툽 (MAKTUB)마크툽 (MAK
27위 | 천상연 - 이창섭이창섭
28위 | 나는 반딧불 - 황가람황가람
29위 | 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지 - AK

### 5) 네이버 지식인으로 검색(open API 사용X)

In [17]:
# 방법1
from requests import get
from bs4 import BeautifulSoup
keyword = '쳇지피티'
url = 'https://kin.naver.com/search/list.naver'
params = {'query':keyword}
response = get(url, params=params)
# url = f'https://kin.naver.com/search/list.naver?query={keyword}'
print(response.status_code)
soup = BeautifulSoup(response.text, "html.parser")
# soup

200


In [14]:
# 방법2
from urllib.request import urlopen
from urllib.parse import quote
keyword = quote('쳇지피티')
url = f'https://kin.naver.com/search/list.naver?query={keyword}'
response = urlopen(url)
print(response.status)
soup = BeautifulSoup(response, 'html.parser')
#soup

200


In [26]:
# 페이징
from urllib.request import urlopen
from urllib.parse import quote
from bs4 import BeautifulSoup
import pandas as pd
keyword = quote('쳇지피티')
pages = 3
items_list = [] # 데이터list(title, link)
for page in range(1, pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    #print(url)
    response = urlopen(url)
    #print(response.status)
    soup = BeautifulSoup(response, 'html.parser')
    items = soup.select('dt > a')
    for item in items:
        items_list.append({
            'title': item.text,
            'link' : item.attrs.get('href')
        })
        #title = item.text
        #link = item.attrs.get('href')
        #print( title, link)
df = pd.DataFrame(items_list)
df.head(1)

,title,link
0,쳇지피티 사주 정확성,https://kin.naver.com/qna/detail.naver?d1id=3&...


## 2.2 openAPI사용 : json 웹데이터 수집
### 1) 네이버 지식인으로 검색(open API사용 o)
 - 네이버 개발자센터에서 애플리케이션등록(이름/검색/WEB설정http://localhost)
 - .env파일에 CLIENT_ID/CLIENT_SECRET 환경변수 저장
 - 환경변수를 읽기 위해서 pip install dotenv
 - 특정 keyword를 지식검색(데이터수 30개)

In [ ]:
# %pip install dotenv

In [1]:
# 환경변수 읽기
from dotenv import load_dotenv
import os
load_dotenv() # 현 소스와 같은 폴더내의 .env를 메모리에 load #dotenv_path='.env'
print(os.getenv('CLIENT_ID'))
print(os.getenv('CLIENT_SECRET'))

yVma_tQ9jdjwwtSBal04
QYzcDVdeEr


In [41]:
# 네이버 검색 API 예제 - 블로그 검색
import os
import urllib.request
import json
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = urllib.parse.quote("한성키보드")
url = "https://openapi.naver.com/v1/search/kin.json?query=" + encText # JSON 결과 - 지식인
# url = "https://openapi.naver.com/v1/search/blog?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
#headers = {
#    'X-Naver-Client-Id':client_id,
#    'X-Naver-Client-Secret':client_secret
#}
#request = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{
	"lastBuildDate":"Tue, 11 Nov 2025 11:48:41 +0900",
	"total":49156,
	"start":1,
	"display":10,
	"items":[
		{
			"title":"<b>한성키보드<\/b> 키 바뀜",
			"link":"https:\/\/kin.naver.com\/qna\/detail.naver?d1id=1&dirId=1010402&docId=486531042&qb=7ZWc7ISx7YKk67O065Oc&enc=utf8",
			"description":"<b>한성키보드<\/b> 왼쪽 컨트롤이랑 캡스락이랑 바껴서 입력이되는데 다시 컨트롤을 컨트롤로 어떻게 바꾸나요 ㅜㅜ 컨트롤을 누르면 캡스락이눌리고 캡스락을누르면 컨트롤로 되고 .. 급합니다ㅠㅜ <b>한성 키보드<\/b>를 사용하시다가 키가... "
		},
		{
			"title":"<b>한성키보드<\/b> 질문합니다",
			"link":"https:\/\/kin.naver.com\/qna\/detail.naver?d1id=1&dirId=1010402&docId=484470395&qb=7ZWc7ISx7YKk67O065Oc&enc=utf8",
			"description":"... 저 가격만큼의 차이가 있는지 알려주세요~ 혹시 그리고 <b>한성<\/b>컴퓨터 외에 키감 좋고 예쁜 <b>키보드<\/b> 있으면 추천해주셔도 좋아요 &lt;1번 <b>한성<\/b>컴퓨터 GK898B PRO 염료승화 유무선 <b>키보드<\/b>&gt; &lt;2번 <b>한성<\/b>컴퓨터 TGF cloud CF&gt; 가격만큼 차이가... "
		},
		{
			"title":"<b>한성키보드<\/b>랑 맥북이 완벽호환이 안돼요",
			"link":"https:\/\/kin.naver.com\/qna\/detail.naver?d1id=1&dirId=1010402&docId=485437646&qb=7ZWc7ISx7YKk67O065Oc&enc=utf8",
			"descrip

In [51]:
import os
import requests
import pandas as pd
import json # response텍스트를 json스타일의 딕셔너리로
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = "쳇지피티"
url = "https://openapi.naver.com/v1/search/kin" # JSON 결과
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
params = {'query':encText,
          'display':30
         }
response = requests.get(url, params=params, headers=headers)
rescode = response.status_code
if(rescode==200):
    #response_body = json.loads(response.text)['items'] #딕셔너리로 변경
    items = response.json()['items']
    items_list = []
    for item in items:
        title = item['title'].replace('<b>','').replace('</b>','')
        link = item.get('link')
        description = item.get('description').replace('<b>','').replace('</b>','')
        items_list.append([title, link, description])
else:
    print("Error Code:" + rescode)
    
pd.DataFrame(items_list, columns=['title', 'link', 'description']).head()

,title,link,description
0,쳇지피티 사주 정확성,https://kin.naver.com/qna/detail.naver?d1id=3&...,쳇지피티 사주 정확성 몇프로 정도 될까요? 큰틀은 얼추 맞는거 같기도하고 모르겠네요...
1,쳇지피티 관해서,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티한테 질문 제대로 하려면 어떻게 해야 하냐요?... 쳇지피티 한테 하는질문도...
2,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지),https://kin.naver.com/qna/detail.naver?d1id=3&...,사주에서 뭐가 부족한지 봐주세요(쳇지피티 답변 금지) 남매입니다 남자 1986년 9...
3,쳇지피티 유료로 전환시,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 무료버전으로 카톡대화창 전송해서 상대의심리에 관한 분석요청을 많이했는데 첨...
4,쳇지피티 3일 리셋,https://kin.naver.com/qna/detail.naver?d1id=8&...,... 쳇지피티 어떻게 하는건가욤 유료결제인가요 유료결제는 얼마인가요 그거 하게되면...


### quiz) 네이버 open API를 이용해서 청바지 이미지 100건의 데이터를 ch14_청바지.csv출력
제목,링크,썸네일,sizeheight,sizewidth

In [ ]:
# image
rss/channel/item/title	string	이미지가 검색된 문서의 제목
rss/channel/item/link	string	이미지의 URL
rss/channel/item/thumbnail	string	섬네일 이미지의 URL
rss/channel/item/sizeheight	string	이미지의 세로 크기(단위: 픽셀)
rss/channel/item/sizewidth	string	이미지의 가로 크기(단위: 픽셀)

In [9]:
import os
import requests
import pandas as pd
import json # response텍스트를 json스타일의 딕셔너리로
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = "청바지"
url = "https://openapi.naver.com/v1/search/image" # JSON 결과
headers = {
    'X-Naver-Client-Id':client_id,
    'X-Naver-Client-Secret':client_secret
}
params = {'query':encText,
          'display':100
         }
response = requests.get(url, params=params, headers=headers)
items = response.json()['items']
items_list = []
rescode = response.status_code
if(rescode==200):
    for item in items:
        title = item.get('title')
        link = item['link']
        thumbnail = item['thumbnail']
        sizeheight = item['sizeheight']
        sizewidth = item['sizewidth']
        items_list.append([title, link, thumbnail, sizeheight, sizewidth])
else:
    print("Error Code:" + rescode)
    
df = pd.DataFrame(items_list, columns=['title', 'link', 'thumbnail', 'sizeheight', 'sizewidth'])
df
df.to_csv('data/bluejin.csv')

In [11]:
import pandas as pd
ss = pd.read_csv('data/bluejin.csv')
ss

,Unnamed: 0,title,link,thumbnail,sizeheight,sizewidth
0,0,여성 하이웨이스트 연청 와이드 청바지 통바지 편안함 스타일리시 캐주얼 데님 : 믿음...,https://shop-phinf.pstatic.net/20251015_281/17...,https://search.pstatic.net/common/?type=b150&s...,1000,1000
1,1,-30 여자 청바지 여성 데님 밴딩 일자 통 와이드 팬츠,http://shopping.phinf.naver.net/main_5666884/5...,https://search.pstatic.net/common/?type=b150&s...,600,600
2,2,키작녀청바지 세미와이드청바지 진청바지 : 미니팬츠,http://shop1.phinf.naver.net/20240320_292/1710...,https://search.pstatic.net/common/?type=b150&s...,1000,1000
3,3,작은 자른 슬릿 높은 허리 청바지 2022 봄 여름 얇은 스트레이트 느슨한 모든 일...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://search.pstatic.net/sunny/?type=b150&sr...,492,492
4,4,JEEP spirit 남성 청바지 스판 남자 캐쥬얼 팬츠 지프 : 우수한 편의점,https://shop-phinf.pstatic.net/20251014_131/17...,https://search.pstatic.net/common/?type=b150&s...,800,800
...,...,...,...,...,...,...
95,95,남자청바지 럭셔리 빅사이즈 여름 스판 조거팬츠 청바지 비즈니스 라이트 k25 : 심...,https://shop-phinf.pstatic.net/20250521_137/17...,https://search.pstatic.net/common/?type=b150&s...,800,800
96,96,발망청바지 | 트렌비,https://image-cdn.trenbe.com/productmain-image...,https://search.pstatic.net/sunny/?type=b150&sr...,900,900
97,97,남성 겨울청바지 탈부착 오리털 패딩팬츠 청바지 : HILLZO-URBAN,http://shop1.phinf.naver.net/20231125_290/1700...,https://search.pstatic.net/common/?type=b150&s...,790,790
98,98,남자 큰옷 빅사이즈 BS 롱 와이드통 커브 고무밴딩 청바지 1089 (블루청) : ...,http://shop1.phinf.naver.net/20250930_287/1759...,https://search.pstatic.net/common/?type=b150&s...,1000,1000


In [42]:
def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법2)'
    from urllib.request import urlopen, Request
    from urllib.parse import quote
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    encText = quote(query)
    url = f'https://openapi.naver.com/v1/search/image?query={encText}&display=100'
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    request = Request(url, headers=headers)
    response = urlopen(request)
    # print(response.read().decode('utf-8'))
    items = json.loads(response.read().decode('utf-8'))['items']
    items_list = []
    for item in items:
        #print(item)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
    return pd.DataFrame(items_list)
df = get_image_list("청바지")
#df.to_csv('data/ch14_청바지.csv')
#df.sample()

In [4]:
def get_image_list(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, display:100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    #print(items[0])
    items_list = []
    for idx, item in enumerate(items):
        print(idx)
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
        # 이미지 파일 저장
        save_image('메인', idx, link, query)
        #save_image('썸네일', idx, thumbnail, query)
        # 진행상황 표시
        if (idx%10 == 0) & (idx!=0):
            print(f'= = = {idx}% 진행완료 = = =')
    print('===== 완료 =====')
    return pd.DataFrame(items_list)
#df = get_image_list("청바지")
#df.to_csv('data/ch14_청바지.csv')
#df.sample()

In [26]:
print(df.loc[0, '링크'])
print(df.loc[0, '썸네일'])

https://shop-phinf.pstatic.net/20251015_281/1760512390478oRzer_PNG/42211957144911892_1321802660.png
https://search.pstatic.net/common/?type=b150&src=https://shop-phinf.pstatic.net/20251015_281/1760512390478oRzer_PNG/42211957144911892_1321802660.png


In [8]:
def save_image(attr, idx, link, query):
    'link의 이미지를 image/attr_idx_query.확장자로 local에 저장'
    import os
    import requests
    file_extension = link.split('.')[-1]
    quote_index = file_extension.find('?')
    if(quote_index != -1):
        file_extension = file_extension[:quote_index]
    img = requests.get(link).content
    
    os.makedirs('image', exist_ok=True) # 폴더가 없으면 생성
    
    with open(f'image/{attr}_{idx+1}_{query}.{file_extension}', 'wb') as f:
        f.write(img)

In [56]:
save_image('메인', 0, df.loc[0, '링크'], '청바지')

In [51]:
get_image_list('청바지')

0
1
2
3
4
5
6
7
8
9
===== 완료 =====


,제목,링크,썸네일,sizeheight,sizewidth
0,여성 하이웨이스트 연청 와이드 청바지 통바지 편안함 스타일리시 캐주얼 데님 : 믿음...,https://shop-phinf.pstatic.net/20251015_281/17...,https://search.pstatic.net/common/?type=b150&s...,1000,1000
1,-30 여자 청바지 여성 데님 밴딩 일자 통 와이드 팬츠,http://shopping.phinf.naver.net/main_5666884/5...,https://search.pstatic.net/common/?type=b150&s...,600,600
2,키작녀청바지 세미와이드청바지 진청바지 : 미니팬츠,http://shop1.phinf.naver.net/20240320_292/1710...,https://search.pstatic.net/common/?type=b150&s...,1000,1000
3,작은 자른 슬릿 높은 허리 청바지 2022 봄 여름 얇은 스트레이트 느슨한 모든 일...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://search.pstatic.net/sunny/?type=b150&sr...,492,492
4,JEEP spirit 남성 청바지 스판 남자 캐쥬얼 팬츠 지프 : 우수한 편의점,https://shop-phinf.pstatic.net/20251014_131/17...,https://search.pstatic.net/common/?type=b150&s...,800,800
5,여성 청바지 스트레이트 슬리밍 데님팬츠 여름 찢어진 드레이프 허리 : 리메리아,https://shop-phinf.pstatic.net/20250924_192/17...,https://search.pstatic.net/common/?type=b150&s...,800,800
6,가비진 여성용 스판 밑단 컷팅 일자 청바지 M,http://shopping.phinf.naver.net/main_5144967/5...,https://search.pstatic.net/common/?type=b150&s...,700,700
7,청바지 스판 팬츠 남자명품청바지 데님팬츠 일자핏 럭셔리 : 다바르스토어,http://shop1.phinf.naver.net/20251015_48/17604...,https://search.pstatic.net/common/?type=b150&s...,800,800
8,세미부츠컷 부츠컷청바지 진청데님 진청바지 키작녀청바지 : 우아모르,http://shop1.phinf.naver.net/20251017_197/1760...,https://search.pstatic.net/common/?type=b150&s...,1853,1853
9,"牛仔裤에 있는 光华 白님의 핀 | 여성 청바지, 여성, 청바지",https://i.pinimg.com/736x/be/3e/51/be3e517f638...,https://search.pstatic.net/sunny/?type=b150&sr...,1582,700


In [11]:
def get_image_list_save_image(query):
    'query로 검색한 이미지 정보(정보,링크,썸네일,크기) 100건 데이터 프레임을 return(방법1)'
    import requests
    import json
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    url = 'https://openapi.naver.com/v1/search/image'
    params = {'query':query, 'display':100 }
    headers = {
        'X-Naver-Client-Id':client_id,
        'X-Naver-Client-Secret':client_secret
    }
    response = requests.get(url, params=params, headers=headers)
    #items = response.json()['items']
    items = json.loads(response.text)['items']
    items_list = [] # 정보가 담길 리스트
    for idx, item in enumerate(items):
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        items_list.append({
            '제목':item.get('title'),
            '링크':link,
            '썸네일': thumbnail,
            'sizeheight': int(item.get('sizeheight')),
            'sizewidth':  int(item.get('sizewidth')),
        })
        # 이미지 파일 저장
        save_image('메인', idx, link, query)
        save_image('썸네일', idx, thumbnail, query)
        if (idx%20 == 0) & (idx!=0):
            print(f'= = = {idx}% 진행중 = = =')
    print('= = = 완료 = = =')
    return pd.DataFrame(items_list)
df = get_image_list_save_image("청바지")
df.to_csv('image/ch14_청바지.csv')
df.sample()

= = = 20% 진행중 = = =
= = = 40% 진행중 = = =
= = = 60% 진행중 = = =
= = = 80% 진행중 = = =
= = = 완료 = = =


,제목,링크,썸네일,sizeheight,sizewidth
68,남자 스탠다드 캣 워싱 와이드 데님팬츠 청바지 : 옴므티디,http://shop1.phinf.naver.net/20251017_1/176068...,https://search.pstatic.net/common/?type=b150&s...,1000,1000


In [9]:
get_image_list_save_image('청바지')

= = = 20% 진행완료 = = =
= = = 40% 진행완료 = = =
= = = 60% 진행완료 = = =
= = = 80% 진행완료 = = =


,제목,링크,썸네일,sizeheight,sizewidth
0,여성 하이웨이스트 연청 와이드 청바지 통바지 편안함 스타일리시 캐주얼 데님 : 믿음...,https://shop-phinf.pstatic.net/20251015_281/17...,https://search.pstatic.net/common/?type=b150&s...,1000,1000
1,-30 여자 청바지 여성 데님 밴딩 일자 통 와이드 팬츠,http://shopping.phinf.naver.net/main_5666884/5...,https://search.pstatic.net/common/?type=b150&s...,600,600
2,키작녀청바지 세미와이드청바지 진청바지 : 미니팬츠,http://shop1.phinf.naver.net/20240320_292/1710...,https://search.pstatic.net/common/?type=b150&s...,1000,1000
3,작은 자른 슬릿 높은 허리 청바지 2022 봄 여름 얇은 스트레이트 느슨한 모든 일...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://search.pstatic.net/sunny/?type=b150&sr...,492,492
4,JEEP spirit 남성 청바지 스판 남자 캐쥬얼 팬츠 지프 : 우수한 편의점,https://shop-phinf.pstatic.net/20251014_131/17...,https://search.pstatic.net/common/?type=b150&s...,800,800
...,...,...,...,...,...
95,남자청바지 럭셔리 빅사이즈 여름 스판 조거팬츠 청바지 비즈니스 라이트 k25 : 심...,https://shop-phinf.pstatic.net/20250521_137/17...,https://search.pstatic.net/common/?type=b150&s...,800,800
96,발망청바지 | 트렌비,https://image-cdn.trenbe.com/productmain-image...,https://search.pstatic.net/sunny/?type=b150&sr...,900,900
97,남성 겨울청바지 탈부착 오리털 패딩팬츠 청바지 : HILLZO-URBAN,http://shop1.phinf.naver.net/20231125_290/1700...,https://search.pstatic.net/common/?type=b150&s...,790,790
98,남자 큰옷 빅사이즈 BS 롱 와이드통 커브 고무밴딩 청바지 1089 (블루청) : ...,http://shop1.phinf.naver.net/20250930_287/1759...,https://search.pstatic.net/common/?type=b150&s...,1000,1000


## 2.3 XML 웹데이터 수집
 - RSS / open API 을 통한 XML 웹데이터 수집
### 1) 전국 날씨 RSS를 BeautifalSoup을 이용한 크롤링
 - 구글에 "기상청 RSS" 검색한 첫번째 사이트 이동

In [20]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
items_list = []
url = 'https://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20251106.xml'
# 방법1
# target = requests.get(url)
# soup = BeautifulSoup(target.text, "xml") # pip install lxml 필요없음
# 방법2
target = urlopen(url)
soup = BeautifulSoup(target, "xml")
local_tas = soup.select('local_ta')
for local_ta in local_tas:
    local_name = local_ta.select_one('local_ta_name').text
    week1_normalYear = local_ta.select_one('week1_local_ta_normalYear').text
    week1_similarRange = local_ta.select_one('week1_local_ta_similarRange').text
    week1_minVal = local_ta.select_one('week1_local_ta_minVal').text
    week1_similarVal = local_ta.select_one('week1_local_ta_similarVal').text
    week1_maxVal = local_ta.select_one('week1_local_ta_maxVal').text
    items_list.append({
        '지역':local_name,
        '1주평년기온':week1_normalYear,
        '1주기온범위':week1_similarRange,
        '1주낮을확률':week1_minVal,
        '1주비슷할확률':week1_similarVal,
        '1주높을확률':week1_maxVal
    })
df = pd.DataFrame(items_list)
df['1주평년기온'] = df['1주평년기온'].astype(np.float64)
df['1주낮을확률'] = df['1주낮을확률'].astype(np.int16)
df['1주비슷할확률'] = df['1주비슷할확률'].astype(np.int16)
df['1주높을확률'] = df['1주높을확률'].astype(np.int16)
df.head()

,지역,1주평년기온,1주기온범위,1주낮을확률,1주비슷할확률,1주높을확률
0,"전국(제주도,북한제외)",5.9,5.0~6.8,20,40,40
1,서울ㆍ인천ㆍ경기도,4.8,3.7~5.9,20,40,40
2,강원도 영서,2.6,1.5~3.7,20,40,40
3,강원도 영동,5.8,4.9~6.7,20,40,40
4,대전ㆍ세종ㆍ충청남도,5.4,4.5~6.3,20,40,40


### 2) XML공급하는 open API
 - data.go.kr에서
  - 서울특별시_노선정보조회 서비스(버스ID, 정류장목록과정류장ID)
  - 서울특별시_버스위치정보조회 서비스(실시간 버스 위치 목록)

In [1]:
# step1 버스번호의 버스 ID 받아오기
# 서울특별시_노선정보조회 서비스 : 3번 기능(getBusRouteList)이용
import os
from dotenv import load_dotenv
load_dotenv()
pub_key = os.getenv('PUB_KEY')
print(pub_key)

48b1f5858aa398b5b7e02884367d0d3d2ec4fbe2b53f18c18bf07d11a5b80565


In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
from urllib.parse import quote
import os
pub_key = os.getenv('PUB_KEY')
busNum = '5528'
key = pub_key
url = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
saveFileName = 'data/ch14_busInfo.xml'
print(url)
urlretrieve(url, saveFileName)
with open('data/ch14_busInfo.xml', encoding='utf-8') as f:
    xml = f.read()
xml

http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey=48b1f5858aa398b5b7e02884367d0d3d2ec4fbe2b53f18c18bf07d11a5b80565&strSrch=5528


'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><ServiceResult><comMsgHeader/><msgHeader><headerCd>0</headerCd><headerMsg>정상적으로 처리되었습니다.</headerMsg><itemCount>0</itemCount></msgHeader><msgBody><itemList><busRouteAbrv>5528</busRouteAbrv><busRouteId>100100263</busRouteId><busRouteNm>5528</busRouteNm><corpNm>신인운수  02-854-8001</corpNm><edStationNm>사당역</edStationNm><firstBusTm>20251112043000</firstBusTm><firstLowTm>20240201000000</firstLowTm><lastBusTm>20251112225000</lastBusTm><lastBusYn> </lastBusYn><lastLowTm>              </lastLowTm><length>36.73</length><routeType>4</routeType><stStationNm>가산동</stStationNm><term>12</term></itemList></msgBody></ServiceResult>'

In [3]:
import requests
from urllib.parse import quote
# busNum = quote('구로09')
busNum = '5528'
key = os.getenv('PUB_KEY')
url1 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
response = requests.get(url1)
soup = BeautifulSoup(response.text, 'xml')

for item in soup.select('itemList'):
    busRouteNm = item.select_one('busRouteNm').text
    #print(busRouteNm)
    if busRouteNm == busNum:
        busRouteId = item.select_one('busRouteId').text
        #print(busRouteId)
        break
print('정류소ID :' , busRouteId)

정류소ID : 100100263


In [ ]:
# step2. 버스id로 버스정류장목록받아오기(정류장명, 정류장id, 경도, 위도)
# 서울특별시_노선정보조회 서비스 - 4번 기능(getStaionsByRouteList)

In [4]:
import pandas as pd
url2 = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}'
response = requests.get(url2)
soup = BeautifulSoup(response.content, 'xml')
itemLists = soup.find_all('itemList')
print(f'{busNum}번 정류장 갯수 : {len(itemLists)}')
bus_station = []
for itemList in itemLists:
    stationNm = itemList.find('stationNm').text # 정류소 이름
    station   = itemList.find('station').text   # 정류소 id
    gpsX      = itemList.find('gpsX').text      # 경도
    gpsY      = itemList.find('gpsY').text      # 위도
    bus_station.append([stationNm, station, gpsX, gpsY])
df_station = pd.DataFrame(bus_station, columns=['정류소명', 'id', '경도', '위도'])
df_station.head()

5528번 정류장 갯수 : 82


,정류소명,id,경도,위도
0,가산동기점,117000130,126.8752549755,37.4852270318
1,한일유엔아이아파트,117000320,126.8762259351,37.4854590227
2,디지털3단지.롯데이노베이트,117000016,126.8767176309,37.484867474
3,디지털3단지운동장,117000019,126.8778930341,37.4821627292
4,디지털3단지.(주)로옴코리아,117000023,126.88022,37.479611


In [ ]:
# step3. 버스id로 실시간위치정보를 받아오기(차량번호, 혼잡도, 최종정류장id, 다음정류장id, 도착소요시간)
# 서울특별시_버스위치정보조회 서비스 - 2번(getBusPosByRtidList)

In [8]:
url3 = f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?serviceKey={key}&busRouteId={busRouteId}'
# print(url3)
response = requests.get(url3)
soup = BeautifulSoup(response.text, 'xml')
itemLists = soup.select('itemList')
print(f'운행중인 버스는 {len(itemLists)}대입니다')
bus_position = []
for itemList in itemLists:
    plainNo = itemList.select_one('plainNo').text # 차량번호
    congetion = itemList.select_one('congetion').text # 혼잡도(congetion)
    # 0 : 없음, 3 : 여유, 4 : 보통, 5 : 혼잡, 6 : 매우혼잡
    congetion = '없음' if congetion=='0' \
            else '여유' if congetion=='3' \
            else '보통' if congetion=='4' \
            else '혼잡' if congetion=='5' else '매우혼잡'
    gpsX = itemList.select_one('gpsX').text # 경도
    gpsY = itemList.select_one('gpsY').text # 위도
    lastStnId = itemList.select_one('lastStnId').text # 최종정류소id
    nextStId = itemList.select_one('nextStId').text # 다음정류소id
    nextStTm = itemList.select_one('nextStTm').text # 다음정류소까지 소요시간
    
    bus_position.append({
        '차량번호':plainNo,
        '혼잡도':congetion,
        '경도':gpsX,
        '위도':gpsY,
        '최종정류소id':lastStnId,
        '다음정류소id':nextStId,
        '도착소요시간':nextStTm
    })
df_position = pd.DataFrame(bus_position)
df_position.head()

운행중인 버스는 17대입니다


,차량번호,혼잡도,경도,위도,최종정류소id,다음정류소id,도착소요시간
0,서울74사5562,여유,126.877444,37.483328,117000016,117000021,434
1,서울74사5544,여유,126.875243,37.490771,116000141,120000018,1845
2,서울74사5535,여유,126.885206,37.482868,116000147,120000018,1301
3,서울74사5530,여유,126.902452,37.479465,117000033,120000018,594
4,서울74사5526,여유,126.928063,37.483818,120000018,120000457,149


In [10]:
df_station.loc[df_station['id']=='117000016','정류소명'].iloc[0]

'디지털3단지.롯데이노베이트'

In [11]:
df_station.loc[df_station['id']=='117000021','정류소명'].iloc[0]

'가산디지털단지역'

In [13]:
def station_name(row):
    'row의 최종정류소명과 다음정류소명을 추가하고, 도착소용시간을 분으로 바꿔 return'
    row['최종정류소명'] = df_station.loc[df_station['id']==row['최종정류소id'],'정류소명'].iloc[0]
    row['다음정류소명'] = df_station.loc[df_station['id']==row['다음정류소id'],'정류소명'].iloc[0]
    return row
station_name(df_position.loc[0])

차량번호            서울74사5562
혼잡도                    여유
경도             126.877444
위도              37.483328
최종정류소id         117000016
다음정류소id         117000021
도착소요시간                434
최종정류소명     디지털3단지.롯데이노베이트
다음정류소명           가산디지털단지역
Name: 0, dtype: object

In [14]:
def station_name(row):
    row = row.copy()
    'row의 최종정류소명과 다음정류소명을 추가하고, 도착소요시간을 분으로 바꿔 return'
    row['이전정류소명'] = df_station.loc[df_station['id']==row['최종정류소id'],'정류소명'].iloc[0]
    row['다음정류소명'] = df_station.loc[df_station['id']==row['다음정류소id'],'정류소명'].iloc[0]
    minite = int(row['도착소요시간'])//60 # //몫연산자
    sec    = int(row['도착소요시간']) % 60
    row['도착소요시간'] = f'{minite}분{sec}초'
    return row
#station_name(df_position.loc[20])

In [15]:
df = df_position.apply(station_name, axis=1)
df.drop(['최종정류소id','다음정류소id'], axis=1).head()

,차량번호,혼잡도,경도,위도,도착소요시간,이전정류소명,다음정류소명
0,서울74사5562,여유,126.877444,37.483328,7분14초,디지털3단지.롯데이노베이트,가산디지털단지역
1,서울74사5544,여유,126.875243,37.490771,30분45초,구일우성아파트,신림사거리.신림역
2,서울74사5535,여유,126.885206,37.482868,21분41초,구일우성아파트,신림사거리.신림역
3,서울74사5530,여유,126.902452,37.479465,9분54초,가산초등학교,신림사거리.신림역
4,서울74사5526,여유,126.928063,37.483818,2분29초,신림사거리.신림역,서원동승리교앞


# 3절 연습문제(Quiz1)
 - yes24의 베스트셀러 정보를 제공하는 사이트에서 베스트셀러 정보를 수집해서 파일에 저장하세요
  - 베스트셀러 정보 수집 주소 : http://www.yes24.com/24/category/bestseller (1페이지만 추출)
  - https://www.yes24.com/product/category/bestseller?pageNumber={page} (여러 페이지 추출)
  ch14_yes24.csv나 ch14_yes24.txt의 내용(구분문자를 ,나 \t,....)
  - 순위,책이름,저자및출판사,가격
    - 1,[도서]트렌드코리아2026,"김난도,전미영,최지혜,권정윤,한다혜 | 미래의창",18,000원
    - 24,[만화]그 비스트 어쩌구,후쿠다신이치글그림/박연지역 | 소미미디어,25,200원

In [100]:
# 방법1
from requests import get
from bs4 import BeautifulSoup
url = 'http://www.yes24.com/24/category/bestseller'
response = get(url)
#print(response.status_code)
soup = BeautifulSoup(response.text, "html.parser")
book_list = soup.select('div.itemUnit > div.item_info')
book_save = []
for idx, book in enumerate(book_list):
    authlist = []
    name = book.select_one('a.gd_name').text
    auth = book.select_one('span.authPub > a').text
    AuthList = book.select('span.moreAuthLiCont > ul > li > a')
    comp = book.select_one('span.info_pub > a').text
    price = book.select_one('strong.txt_num').text
    if(AuthList != None):
        for auth2 in AuthList:
            authlist.append(auth2.text)
            #print('+++++', auth2.attrs['author'])
        auth2 = ','.join(authlist)
    else:
        auth
    
    if(auth2 != ''):
        auth = auth2
    #print(idx, name, auth, comp, price)
    book_save.append({'순위':idx+1, '책이름':name, '저자':auth, '출판사':comp, '가격':price})
    
df = pd.DataFrame(book_save)
df.to_csv('data/ch14_yes24.csv')

In [ ]:
# import
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re

In [ ]:
# 방법1
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    #print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    # 순위
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks =  [int(ranks_el.text) for ranks_el in ranks_els]
    # 책제목
    titles_els=soup.select("div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    # 저자
    authors_els = soup.select('span.authPub.info_auth')
    authors = [authors_el.text.strip() for authors_el in authors_els]
    # authors = [authors_el.get_text(strip=True) for authors_el in authors_els]
#     print(authors)
    for i, author in enumerate(authors):
        match = re.search(r"(.*)외 \d+명", author)
        if(match):
            authors[i] = match.group()
            print(i, '번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름')
    # 출판사
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    # 가격
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, author, publisher, price in zip(ranks, titles, authors, publishers, prices):
            # print("{:02},{},{} | {},{}".format(rank, title, author, publisher, price))
            f.write(f'{rank}, {title}, {author} | {publisher}, {price}\n')
            bestseller_list.append([rank, title, author, publisher, price])
df = pd.DataFrame(bestseller_list, columns=['순위','책이름','저자','출판사','가격'])
df.to_csv('data/ch14_yes24.csv', index=False)
df.head(1)

In [ ]:
# 방법2
pages = 2
bestseller_list = []
with open('data/ch14_yes24.txt', 'w', encoding='utf-8') as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    #print(url)
    response = urlopen(url)
    soup = BeautifulSoup(response, 'html.parser')
    # 순위
    ranks_els = soup.select('div.img_upper > em.ico.rank')
    ranks =  [int(ranks_el.text) for ranks_el in ranks_els]
    # 책제목
    titles_els=soup.select("div.info_row > a.gd_name")
    titles = [titles_el.text for titles_el in titles_els]
    # 저자
    authors_els = soup.select('span.authPub.info_auth')
    authors = [authors_el.text.split('정보 더 보기/감추기')[0].strip() 
                                   for authors_el in authors_els]
#     for i, author in enumerate(authors):
#         match = re.search(r"(.*)외 \d+명", author)
#         if(match):
#             authors[i] = match.group()
#             print(i, '번째 도서의 저자가 많아 외 x명 뒤에도 길어 그 앞까지 짜름')
    # 출판사
    publishers_els = soup.select('div.info_row.info_pubGrp > span.authPub.info_pub')
    publishers = [publishers_el.text for publishers_el in publishers_els]
    # 가격
    prices_els = soup.select("div.info_row > strong.txt_num")
    prices = [prices_el.text for prices_el in prices_els]
    # print(len(ranks), len(titles), len(writers), len(publishers), len(prices))
    with open('data/ch14_yes24.txt', 'a', encoding='utf-8') as f:
        for rank, title, author, publisher, price in zip(ranks, titles, authors, publishers, prices):
            # print("{:02},{},{} | {},{}".format(rank, title, author, publisher, price))
            f.write(f'{rank}, {title}, {author} | {publisher}, {price}\n')
            bestseller_list.append([rank, title, author, publisher, price])
df = pd.DataFrame(bestseller_list, columns=['순위','책이름','저자','출판사','가격'])
df.to_csv('data/ch14_yes24.csv', index=False)
df.head(1)